In [ ]:
%load_ext autoreload
%autoreload 2
import os
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
from IPython.display import display, HTML
from typing import List
import numpy as np
from copy import deepcopy
import re
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import pandas as pd
import joblib
# from mprompt.config import RESULTS_DIR
import torch.cuda
import json
from sklearn.linear_model import RidgeCV
import pandas as pd
import re
import sklearn.decomposition
import pyLDAvis
import pyLDAvis.lda_model
import numpy as np
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
import adjustText
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import joblib
import os
from os.path import join
from tqdm import tqdm
from mprompt.config import RESULTS_DIR

HUMAN_EVAL_DIR = join(RESULTS_DIR, 'bert', 'human_eval')
# f'b_l{factor_layer}_i{factor_idx}')
fnames = os.listdir(HUMAN_EVAL_DIR)
evals = []
for fname in fnames:
    fdir = os.listdir(join(HUMAN_EVAL_DIR, fname))[0]
    evals.append(joblib.load(join(HUMAN_EVAL_DIR, fname, fdir, 'eval.pkl')))
df = pd.DataFrame([pd.Series(x) for x in evals])
scores_recovered = joblib.load(join(RESULTS_DIR, 'bert', 'wiki_norm_std', 'wiki_our_syn_perc_std_score.pkl'))
df = df[['factor_layer', 'factor_idx', 'explanation_init_strs', 'score_synthetic_std']]
df['score_synthetic_std'] = df['score_synthetic_std'].apply(lambda x: x[0][0]).astype(float)
df['score_recovered'] = df.apply(lambda x: scores_recovered[x['factor_layer']][x['factor_idx']], axis=1)
df = df.rename(
    columns={
        'explanation_init_strs': 'explanation_human',
        'score_synthetic_std': 'score_human',
    }
)
df['explanation_human'] = df['explanation_human'].apply(lambda x: x[0])

# add bert expl
expls_scores_bert = joblib.load(join(RESULTS_DIR, 'bert', 'wiki_norm_std', 'wiki_our_syn_perc_std_score.pkl'))
expls_bert = joblib.load(join(RESULTS_DIR, 'bert', 'expls_bert.jbl'))
df_bert = pd.DataFrame.from_dict({
    'factor_layer': sum([[i] * 1500 for i in range(13)], []),
    'score': expls_scores_bert.flatten(),
    'expl': expls_bert,
    'factor_idx': sum([list(range(1500)) for i in range(13)], []),
})
df['explanation_method'] = df.apply(
    lambda x: df_bert[(df_bert['factor_layer'] == x['factor_layer']) & \
                      (df_bert['factor_idx'] == x['factor_idx'])]['expl'].values[0],
    axis=1)

df.to_pickle(join(RESULTS_DIR, 'results_bert_human_eval.pkl'))
df.head()

In [ ]:
print(df[['score_human', 'score_recovered']].mean())
# print(df_human[['score_human', 'score_recovered']].sem())

In [ ]:
print('win percentage', (df['score_recovered'] > df['score_human']).mean())

In [ ]:
ks = {
        'factor_layer': 'Factor Layer',
        'factor_idx': 'Factor Index',
        'explanation_human': 'Explanation (Human)',
        'explanation_method': 'Explanation (Ours)',
        'score_human': 'Explanation score (Human)',
        'score_recovered': 'Explanation score (Ours)',
    }
print(df.rename(columns=ks)[list(ks.values())].style.hide(axis='index').format(precision=2).to_latex(hrules=True))